In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys
import dgl
# sys.path.append('.')

from glob import glob
# from configs import *
from tqdm import tqdm
# from utils import *
import numpy as np
# from dataset import *k
# from days import *
from time import time
tqdm.monitor_interval = 0
import torch
import json
from time import time
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

In [7]:
groups = [
    
#     '/beegfs/ua349/logs/slurm_1353164.out',
#     ['Linear', '/beegfs/ua349/logs/linear', [2, 3, 4]],
#     ['GCN', '/beegfs/ua349/logs/gcn', [2, 3, 4]],
#     ['MXRNN', '/beegfs/ua349/logs/mxrnn', [2, 3, 4]],
#     ['GCN-LSTM', '/beegfs/ua349/logs/lgcn', [2, 3, 4]],
#     ['ASTGCN', '/beegfs/ua349/logs/astgcn', [1, 2, 3, 4]],
     ['MPRNN', '/beegfs/ua349/logs/mprnn', [1,]],
]

In [8]:
latex = False
delim = ' & ' if latex else ' '
for name, folder, hops in groups:
    hop_avg = []
    for hop in range(1, 6):
        if hop in hops:
            avg = []
            for logfl in glob('%s/*n%d*' % (folder, hop)):
#                 print(logfl)
                with open(logfl) as fl:
                    res = json.load(fl)
                hist, best_eval = res
                if name == 'ASTGCN': best_eval *=2
                avg.append(np.sqrt(best_eval))
            var = np.sqrt(np.var(avg))
            avg = np.mean(avg)
            hop_avg.append((avg, np.sqrt(var)))
        else:
            hop_avg.append(None)
    style = lambda stat: '$%.3f \mypm %.1f$' % (stat[0], stat[1]) if latex else '%.3f' % stat[0]
    out = delim.join([
        name,
        
        delim.join([style(stat) 
                    if stat is not None else '-' 
                        for stat in hop_avg]),
    ])
    print(out)

MPRNN nan - - - -


/home/ua349/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3506: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/ua349/.local/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/ua349/.local/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ua349/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ua349/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
compare = [
    ['MPRNN', '/beegfs/ua349/logs/lgcn', 4,],
    ['ASTGCN', '/beegfs/ua349/logs/astgcn', 4,]
]
compare

[['MPRNN', '/beegfs/ua349/logs/lgcn', 4],
 ['ASTGCN', '/beegfs/ua349/logs/astgcn', 4]]

In [49]:
lookup = {}
for group, folder, hops in compare:
    for logfl in glob('%s/*n%d*' % (folder, hops)):
#         print(group, logfl)
        name = logfl.split('/')[-1].split('_')[0]
        
        if name not in lookup:
            lookup[name] = []
        with open(logfl) as fl:
            res = json.load(fl)
        _, best_eval = res
        lookup[name].append(np.sqrt(best_eval))
# lookup
pairs = list(filter(lambda pair: len(pair[1])==2, lookup.items()))
pairs = sorted(pairs, key=lambda ent: abs(ent[1][0]-ent[1][1]))
list(map(lambda ent: (ent[0], (ent[1][0], ent[1][1]), abs(ent[1][0]-ent[1][1])), pairs))

[('403894-403895',
  (0.18682624167202205, 0.13070161940042763),
  0.056124622271594427),
 ('402505-402506',
  (0.1742053901015929, 0.11542917684811484),
  0.05877621325347805),
 ('404824-401310',
  (0.22574220646676776, 0.15851870810857877),
  0.067223498358189),
 ('402003-402004',
  (0.2254299871112207, 0.15704707067500256),
  0.06838291643621813),
 ('402517-402518',
  (0.22507294546294082, 0.14784774447509738),
  0.07722520098784344),
 ('402528-404853',
  (0.22914721757246984, 0.14958449888424008),
  0.07956271868822976),
 ('403110-403111',
  (0.2398373050801756, 0.15959570474705245),
  0.08024160033312314),
 ('400612-400613',
  (0.27800108502295934, 0.19619744065230443),
  0.0818036443706549),
 ('400007-403782',
  (0.25273164631393097, 0.1690529977341154),
  0.08367864857981558),
 ('401271-401272', (0.2456168723920601, 0.16035123755532), 0.0852656348367401),
 ('401045-401988',
  (0.22537156363667804, 0.13922984576103772),
  0.08614171787564032),
 ('400732-403769',
  (0.291957343083